# <center>Ultralytics OBB</center>

在上一节中，我们深入分析了 YOLO 目标检测算法的多个变种，并详细介绍了通过注册 EfficientNMS 插件来统一这些变种输出节点的方法。这种方法不仅简化了输出格式，还显著提升了非极大值抑制（NMS）后处理的效率。在本节中，我们将进一步拓展目标检测技术的边界，探讨一种更为先进的变体——定向边界框（Oriented Bounding Box，简称 OBB）目标检测。我们将深入分析 Ultralytics OBB 的模型结构，并通过注册自定义插件来进一步提升模型的性能。

## <center>Ultralytics OBB 结构分析</center>

定向边界框（Oriented Bounding Box，简称 OBB）目标检测是一种前沿的目标检测技术，它通过引入旋转角度来更精准地定位图像中的物体。与传统的水平边界框（Axis-Aligned Bounding Box，简称 AABB）相比，OBB 能够更紧密地贴合物体的形状，从而在复杂场景中提供更高的检测精度。

Ultralytics OBB 模型的核心优势在于其能够处理旋转的边界框。与 Ultralytics Detect 模型相比，[Ultralytics OBB 模型的输出在每个边界框中多了一个表示旋转角度的参数](https://github.com/ultralytics/ultralytics/blob/v8.3.23/ultralytics/nn/modules/head.py#L217)。这一额外的参数使得模型能够更灵活地适应不同方向的物体，从而显著提升检测的准确性和鲁棒性。

In [ ]:
import netron
from IPython.display import IFrame, display

url, port = netron.start("./course_data/models/OBB/yolo11n-obb.onnx", verbosity=0, browse=False)
display(IFrame(f'http://{url}:{port}', width='100%', height='520px'))

与 Detect 模型类似，OBB 模型的输入节点同样接收一个包含 N 个样本的批次，每个样本是一张具有 3 个颜色通道、尺寸为高 × 宽像素的图像。但在输出节点上存在一些差异。OBB 模型在**检测输出中额外增加了一个维度，用于表示边界框的旋转角度信息**。因此，OBB 模型的输出维度为 `(batch, 5 + classes, predicts)`，其中 `5 + classes` 表示每个锚点的坐标 `(x, y, w, h, angle)` 以及 classes 个类别的置信度。

## <center>Ultralytics OBB 注册自定义插件</center>

在 YOLO Detect 算法的变体中，通过集成 EfficientNMS 插件可以显著提高非极大值抑制（NMS）后处理的效率。然而，这个插件并不适用于 Ultralytics OBB 模型，原因有两点：首先，EfficientNMS 插件的输出节点 `detection_boxes` 不包含旋转角度信息；其次，Detect 和 OBB 在执行 NMS 时对交并比（IOU）的计算方法也有所不同。为了解决这些问题，在之前的章节中我们开发了一个名为 EfficientRotatedNMS 的自定义插件。

由于 EfficientRotatedNMS 插件旨在提高 OBB 模型的非极大值抑制（NMS）后处理效率，因此该插件在参数设置、输入和输出方面将与 EfficientNMS 插件保持一致。不过，输入的 Boxes 与输出的 `detection_boxes` 的最后一个维度的大小从 4 扩展到了 5。这一变化是为了容纳边界框的旋转角度信息，从而确保在 NMS 过程中能够正确处理 OBB 模型的旋转边界框特性。

In [1]:
import onnx
import onnxsim
import torch
from ultralytics import YOLO
from course_functions.head import UltralyticsOBB

def export_model(model_path: str, output_path: str, batch: int = 1, 
                 imgsz: tuple = (640, 640), max_boxes: int = 100, 
                 iou_thres: float = 0.45, conf_thres: float = 0.25, 
                 opset_version: int = 11) -> None:
    """
    将 YOLO 模型导出为 ONNX 格式，并修改检测头。
    
    参数：
    - model_path: YOLO 模型权重文件的路径。
    - output_path: 导出的 ONNX 模型保存路径。
    - batch: 模型的批量大小，默认为 1。
    - imgsz: 模型输入的图像尺寸，默认为 (640, 640)。
    - max_boxes: 每张图片的最大检测数量，默认为 100。
    - iou_thres: NMS 的 IoU 阈值，默认为 0.25。
    - conf_thres: 检测的置信度阈值，默认为 0.25。
    - opset_version: ONNX opset 版本，默认为 11。
    """
    # 加载指定权重的 YOLO 模型并设置为 CPU 模式
    model = YOLO(model=model_path, verbose=False).model.to('cpu')
    
    # 修改 Detect 层的参数
    for m in model.modules():
        if m.__class__.__name__ == "OBB":
            m.__class__ = type("OBB", (UltralyticsOBB,), {
                "dynamic": batch <= 0,  # 是否需要动态批量大小
                "max_det": max_boxes,  # 每张图片的最大检测数量
                "iou_thres": iou_thres,  # NMS 的 IoU 阈值
                "conf_thres": conf_thres,  # 检测的置信度阈值
            })
            break

    # 创建模型的虚拟输入张量
    dummy_input = torch.randn(batch, 3, *imgsz).to('cpu')
    model.eval().float()  # 将模型设置为评估模式，并确保其为浮点精度
    model(dummy_input)  # 使用虚拟输入运行模型，以确保其正确配置

    # 将模型导出为 ONNX 格式
    torch.onnx.export(
        model,
        dummy_input,
        output_path,
        opset_version=opset_version,
        input_names=['images'],
        output_names=["num_dets", "det_boxes", "det_scores", "det_classes"],
        dynamic_axes={
            "images": {0: "batch", 2: "height", 3: "width"},
            "num_dets": {0: "batch"},
            "det_boxes": {0: "batch"},
            "det_scores": {0: "batch"},
            "det_classes": {0: "batch"},
        } if batch <= 0 else None,  # 如果批量大小是动态的，则定义动态轴
    )

    # 加载导出的 ONNX 模型并进行验证
    model_onnx = onnx.load(output_path)
    onnx.checker.check_model(model_onnx)

    # 则更新输出节点维度
    shapes = {
        'num_dets': ["batch" if batch <= 0 else batch, 1],
        'det_boxes': ["batch" if batch <= 0 else batch, max_boxes, 5],
        'det_scores': ["batch" if batch <= 0 else batch, max_boxes],
        'det_classes': ["batch" if batch <= 0 else batch, max_boxes],
    }
    for node in model_onnx.graph.output:
        for idx, dim in enumerate(node.type.tensor_type.shape.dim):
            dim.dim_param = str(shapes[node.name][idx])

    # 简化 ONNX 模型
    model_onnx, check = onnxsim.simplify(model_onnx)
    assert check, "Simplified ONNX model could not be validated"
    onnx.save(model_onnx, output_path)  # 保存简化后的 ONNX 模型

# 调用函数导出模型
export_model("./course_data/models/OBB/yolo11n-obb.pt", "./course_data/models/OBB/yolo11n-obb_with_plugin.onnx")

/home/my_ubuntu/AI_deployment/TRT-TritonServer-Lesson/course_functions/head.py:225: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.shape != shape:
[W707 22:00:31.725027832 shape_type_inference.cpp:1995] Warning: The shape inference of TRT::EfficientRotatedNMS_TRT type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W707 22:00:31.730785611 shape_type_inference.cpp:1995] Warning: The shape inference of TRT::EfficientRotatedNMS_TRT type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
[W707 22:00:31.732053790 shape_type_inference.cpp:1995] Warni

In [ ]:
url, port = netron.start("./course_data/models/OBB/yolo11n-obb_with_plugin.onnx", verbosity=0, browse=False)
display(IFrame(f'http://{url}:{port}', width='100%', height='620px'))

## <center>总结</center>

本节课程深入探讨了 Ultralytics OBB 模型的结构特点及其在目标检测中的应用。我们首先分析了 OBB 模型相较于传统 AABB 模型的优势，特别是在处理旋转物体时的精准定位能力。随后，我们通过注册自定义插件 EfficientRotatedNMS，解决了 OBB 模型在非极大值抑制（NMS）后处理中的特殊需求，进一步提升了模型的性能和效率。通过本节的学习，希望大家能够掌握如何对 OBB 模型进行优化，并在实际项目中应用这些技术，以实现更高效、更精准的目标检测。